# Chapter 3: Bracketing

In [1]:
import numpy as np
from dataclasses import dataclass

## Algorithm 3.1

In [ ]:
def bracket_minimum(f, x=0, s=1e-2, k=2.0):
    a, ya = x, f(x)
    b, yb = a + s, f(a+s)
    
    if yb > ya:
        a, b = b, a
        ya, yb = yb, ya
        s = -s
    
    while True:
        c, yc = b + s, f(b+s)
        if yc > yb:
            return (a,c) if a < c else (c, a)
        a, ya, b, yb = b, yb, c, yc


## Algorithm 3.2

In [54]:
def fibonacci_search(f, a, b, n, epsilon=0.01):
    golden_ratio = 1.61803
    s = (1-np.sqrt(5))/(1+np.sqrt(5))
    rho = 1/golden_ratio*(1-s**(n+1))/(1-s**n)
    d = rho*b + (1- rho)*a
    yd = f(d)

    for i in range(n-1):
        if i == n-1:
            c = epsilon*a + (1-epsilon)*d
        else:
            c = rho*a + (1-rho)*b
        yc = f(c)

        if yc < yd:
            b, d, yd = d, c, yc
        else:
            a, b = b, c
        rho = 1 / (golden_ratio*(1-s**(n-i+1)))/(1-s**(n-i))
    return (a,b) if a<b else (b,a)

## Algorithm 3.3

In [57]:
def golden_section_search(f, a, b, n):
    golden_ratio = 1.61803
    rho = golden_ratio - 1
    d = rho*b + (1-rho)*a
    yd = f(d)
    for i in range(n-1):
        c = rho*a + (1-rho)*b
        yc = f(c)
        if yc < yd:
            b, d, yd = d, c, yc
        else:
            a, b = b,c
    
    return (a,b) if a<b else (b, a)

## Algorithm 3.4

In [42]:
def quadratic_fit_search(f, a, b, c, n):
    ya, yb, yc = f(a), f(b), f(c)
    for i in range(n-3):
        x = 0.5*(ya*(b**2-c**2)+yb*(c**2-a**2)+yc*(a**2-b**2))/(ya*(b-c)+yb*(c-a)+yc*(a-b))
        yx = f(x)
        if x > b:
            if yx > yb:
                c, yc = x, yx
            else:
                a, ya, b, yb = b, yb, x, yx
        elif x < b:
            if yx > yb:
                a, ya = x, yx
            else:
                c, yc, b, yb = b, yb, x, yx
    
    return (a, b, c)

### Example

In [43]:
def func(x):
    return 4 + x*np.cos(x+1)

a = 0.0
b = 2
c = 3.5
n = 20

sol = quadratic_fit_search(func, a, b, c, n)
print(sol)

(1.3333333333333333, 2.519436058565487, 2.5194360660675903)


## Algorithm 3.5

In [26]:
@dataclass
class Pt:
    x: float
    y: float

def _get_sp_intersection(A, B, l):
    t = ((A.y - B.y) - l*(A.x - B.x)) / (2*l)
    return Pt(A.x + t, A.y - t*l)

def shubert_piyavskii(f, a, b, l, epsilon, delta=0.01):
    m = (a+b)/2
    A, M, B = Pt(a, f(a)), Pt(m, f(m)), Pt(b, f(b))
    pts = [A, _get_sp_intersection(A, M, l),
            M, _get_sp_intersection(M, B, l), B]
    Delta = np.inf
    counter = 0
    while Delta > epsilon:
        counter += 1
        i = np.argmin([P.y for P in pts])
        P = Pt(pts[i].x, f(pts[i].x))
        Delta = P.y - pts[i].y

        P_prev = _get_sp_intersection(pts[i-1], P, l)
        P_next = _get_sp_intersection(P, pts[i+1], l)

        del pts[i]
        pts.insert(i, P_next)
        pts.insert(i, P)
        pts.insert(i, P_prev)

    intervals = []
    P_min = pts[2*(np.argmin([P.y for P in pts[::2]])) - 1]
    y_min = P_min.y
    for j in range(1,len(pts),2):
        if pts[j].y < y_min:
            dy = pts[i].y - pts[j].y
            x_lo = max([a, pts[j].x - dy/l])
            x_hi = min([b, pts[j].x + dy/l])
            _empty = len(intervals) == 0
            if (not _empty) and (intervals[-1][1] + delta >= x_lo):
                intervals[-1] = (intervals[-1][0], x_hi)
            else:
                intervals.insert(0,(x_lo, x_hi))
    
    return (pts[i], intervals)

### Example

In [39]:
def func(x):
    return 4 + x*np.cos(x+1)

a = 0.0
b = 3.5
l = 1.7
epsilon = 0.00001

sol = shubert_piyavskii(func, a, b, l, epsilon)
print(sol[1])

[(2.5131687262449933, 2.5287372947013598)]


## Algorithm 3.6

In [77]:
def bisection(f_deriv, a, b, epsilon):
    if a > b:
        a, b = b, a
    
    ya, yb = f_deriv(a), f_deriv(b)

    if ya == 0:
        b = a
    if yb == 0:
        a = b

    while b - a > epsilon:
        x = (a+b)/2
        y = f_deriv(x)
        if y == 0:
            a, b = x, x
        elif y*ya> 0:
            a = x
        else:
            b = x
    
    return (a,b)

In [81]:

def func_deriv(x):
    return np.cos(x+1)-x*np.sin(x+1)

a = 0.5
b = 3.5
epsilon = 0.00001

sol = bisection(func_deriv, a, b, epsilon)
print(sol)

(2.5194358825683594, 2.519441604614258)


## Algorithm 3.7

In [92]:
def bracket_sign_change(f_deriv, a, b, k=2):
    if a > b:
        a, b = b, a

    center, half_width = (b+a)/2, (b-a)/2
    while f_deriv(a)*f_deriv(b) > 0:
        half_width *= k
        a = center - half_width
        b = center + half_width

    return (a, b)